## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,76.96,84,88,10.11,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,41.49,94,17,2.73,few clouds
2,2,Lasa,IT,46.6166,10.7002,65.28,92,100,3.83,overcast clouds
3,3,Vila Velha,BR,-20.3297,-40.2925,80.96,68,20,17.27,few clouds
4,4,Jamestown,US,42.0970,-79.2353,70.30,54,97,5.01,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# use the loc method to filter the city_data_df DataFrame for temperature criteria collected in Step 2, then create a new DataFrame.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,76.96,84,88,10.11,overcast clouds
3,3,Vila Velha,BR,-20.3297,-40.2925,80.96,68,20,17.27,few clouds
14,14,Hithadhoo,MV,-0.6000,73.0833,82.58,74,21,9.62,few clouds
16,16,Aracati,BR,-4.5617,-37.7697,84.60,62,32,14.25,scattered clouds
20,20,Nioro,GM,13.3500,-15.7500,84.81,66,2,2.26,clear sky
21,21,Okakarara,NaN,-20.5833,17.4333,77.83,16,95,19.06,overcast clouds
22,22,Saint George,US,37.1041,-113.5841,86.47,34,1,3.00,clear sky
23,23,Hilo,US,19.7297,-155.0900,79.61,81,78,1.01,light rain
28,28,Kavieng,PG,-2.5744,150.7967,81.39,78,87,10.20,overcast clouds
30,30,Faanui,PF,-16.4833,-151.7500,79.34,74,21,18.43,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Drop all rows with missing information
clean_df = preferred_cities_df.dropna(how='any')

clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,NaN
3,Vila Velha,BR,80.96,few clouds,-20.3297,-40.2925,NaN
14,Hithadhoo,MV,82.58,few clouds,-0.6000,73.0833,NaN
16,Aracati,BR,84.60,scattered clouds,-4.5617,-37.7697,NaN
20,Nioro,GM,84.81,clear sky,13.3500,-15.7500,NaN
22,Saint George,US,86.47,clear sky,37.1041,-113.5841,NaN
23,Hilo,US,79.61,light rain,19.7297,-155.0900,NaN
28,Kavieng,PG,81.39,overcast clouds,-2.5744,150.7967,NaN
30,Faanui,PF,79.34,few clouds,-16.4833,-151.7500,NaN
34,Atuona,PF,77.61,light rain,-9.8000,-139.0333,NaN


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
        # Grab the first hotel from the results and store the name.
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Vila Velha,BR,80.96,few clouds,-20.3297,-40.2925,Quality Suites Vila Velha
14,Hithadhoo,MV,82.58,few clouds,-0.6000,73.0833,Scoop Guest House
16,Aracati,BR,84.60,scattered clouds,-4.5617,-37.7697,Motel Pousada Milleniun - Aracati-CE
22,Saint George,US,86.47,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
...,...,...,...,...,...,...,...
701,Jalu,LY,80.10,clear sky,29.0331,21.5482,صاحبة الفخامة
702,Kieta,PG,77.70,light rain,-6.2167,155.6333,Uruna Bay Retreat
703,Karangampel,ID,79.39,broken clouds,-6.4622,108.4519,ROOFTOP WAG ALL
707,Ascension,US,82.00,moderate rain,30.2169,-90.9001,Highland Inn


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Call the figure to plot the data.
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))